Import and initiate findspark
Then import pyspark

In [ ]:
import findspark
findspark.init("/usr/local/spark")

In [ ]:
import pyspark

Instatiate SparkSession with Hive support

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example") \
        .config("spark.sql.warehouse.dir", "hdfs://localhost:54310/user/hive/warehouse") \
        .enableHiveSupport() \
        .getOrCreate()

In [45]:
spark.sql('show databases').show()

+------------+
|databaseName|
+------------+
|     default|
|        nyse|
|      nyse_1|
|      office|
|    retaildb|
+------------+



In [46]:
spark.sql("use retaildb")

DataFrame[]

In [47]:
spark.sql('show databases').show()


+------------+
|databaseName|
+------------+
|     default|
|        nyse|
|      nyse_1|
|      office|
|    retaildb|
+------------+



In [48]:
spark.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|retaildb| customer|      false|
|retaildb|custstate|      false|
|retaildb|  product|      false|
+--------+---------+-----------+



In [49]:
spark.sql('select * from customer').show()

+-----+----------------+-------------+------+-----+
|  cid|           cname|        ccity|cstate| czip|
+-----+----------------+-------------+------+-----+
|11039|     Mary Torres|       Caguas|    PR|  725|
| 5623|      Jose Haley|     Columbus|    OH|43207|
| 5829|      Mary Smith|      Houston|    TX|77015|
| 6336|  Richard Maddox|       Caguas|    PR|  725|
| 1708|  Margaret Booth|    Arlington|    TX|76010|
|10227|  Mary Henderson|       Caguas|    PR|  725|
|  839|     Lisa Walker|       Caguas|    PR|  725|
| 7604|   Jonathan Hill|      Phoenix|    AZ|85040|
| 6485|Carolyn Sheppard|Pompano Beach|    FL|33063|
| 4737|    Mary Mendoza|       Caguas|    PR|  725|
| 5973|   Michael Smith|       Caguas|    PR|  725|
| 9205|    James Holmes|     Hilliard|    OH|43026|
|  138|     Mary Dawson|       Caguas|    PR|  725|
|  371|    Adam Marquez|  San Antonio|    TX|78223|
| 9285|    Gloria Smith|       Caguas|    PR|  725|
| 1209|       Mary Webb|   San Marcos|    TX|78666|
| 3021|  Nan

In [50]:
spark.sql('select * from product').show()

+------+--------------------+------------------+------+------+------+-----+
|   pid|               pname|         pcategory|pcatid|pprice|pquant|  cid|
+------+--------------------+------------------+------+------+------+-----+
| 98660|O'Brien Men's Neo...|           Fishing|    45| 49.98|     2|12080|
| 97749|O'Brien Men's Neo...|           Fishing|    45| 49.98|     2|12279|
|103889|Under Armour Wome...|      Boxing & MMA|    12| 31.99|     4| 3343|
| 22751|O'Brien Men's Neo...|           Fishing|    45| 49.98|     5|12382|
|129001|Pelican Sunstream...|           Boating|    47|199.99|     1| 5032|
|165110|Nike Men's CJ Eli...|            Cleats|    17|129.99|     1| 2663|
|140220|Diamondback Women...| Bike & Skate Shop|    42|299.98|     1|  702|
| 77426|Field & Stream Sp...|Hunting & Shooting|    44|399.98|     1| 3959|
|101870|Perfect Fitness P...|   As Seen on  TV!|    16| 59.99|     4| 1836|
|131685|Nike Men's CJ Eli...|            Cleats|    17|129.99|     1| 5212|
|164177|Peli

In [51]:
custstate= spark.sql (" select cstate, count(*) as statecount from customer group by cstate having statecount>=10")

In [52]:
custstate.show()

+------+----------+
|cstate|statecount|
+------+----------+
|    AZ|        19|
|    NJ|        19|
|    VA|        14|
|    MI|        28|
|    NV|        16|
|    CA|       187|
|    NC|        19|
|    MD|        19|
|    MO|        13|
|    IL|        40|
|    OH|        29|
|    TN|        12|
|    PA|        28|
|    NY|        79|
|    TX|        62|
|    GA|        14|
|    MA|        12|
|    FL|        32|
|    CO|        12|
|    PR|       505|
+------+----------+
only showing top 20 rows



In [53]:
prd = spark.sql("SELECT cid, pcategory, count(*) as prdcount FROM product WHERE pprice>20.0 GROUP BY cid, pcategory order by pcategory ")

In [54]:
prd.show()

+-----+-----------+--------+
|  cid|  pcategory|prdcount|
+-----+-----------+--------+
| 1987|Accessories|       1|
| 1615|Accessories|       1|
| 1398|Accessories|       1|
| 3692|Accessories|       1|
|  869|Accessories|       1|
| 4585|Accessories|       1|
| 9653|Accessories|       1|
| 5190|Accessories|       1|
| 4995|Accessories|       1|
| 8722|Accessories|       1|
|10887|Accessories|       1|
|  517|Accessories|       1|
| 8930|Accessories|       1|
| 3159|Accessories|       1|
|10735|Accessories|       1|
| 3614|Accessories|       1|
| 1532|Accessories|       1|
|11745|Accessories|       1|
|  494|Accessories|       1|
| 5337|Accessories|       1|
+-----+-----------+--------+
only showing top 20 rows



Get the list of customers and product categories in which they bought multiple items (quantity) that are more expensive than 200.00 
cname pcategory                 prdcount 
David Smith Hunting & Shooting  2


In [55]:
custlist = spark.sql("SELECT a.cname, b.pcategory, count(*) as prdcount FROM customer a INNER JOIN product b ON a.cid=b.cid WHERE b.pprice>200.00 GROUP BY a.cname, b.pcategory HAVING prdcount>20")

In [56]:
custlist.show()

+----------+------------------+--------+
|     cname|         pcategory|prdcount|
+----------+------------------+--------+
|Mary Smith| Bike & Skate Shop|      22|
|Mary Smith|Hunting & Shooting|      24|
+----------+------------------+--------+



These dataframes can be written to Hive tables after creating a new database. The following lines show the syntax. Plesae note that the Hive metastore version should be compatible with the Spark version to be able to create databases and tables in Hive.

In [58]:
custstate.coalesce(1).write.saveAsTable("retaildb.custstate1")

In [60]:
spark.sql("show tables").show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|retaildb|  customer|      false|
|retaildb| custstate|      false|
|retaildb|custstate1|      false|
|retaildb|   product|      false|
+--------+----------+-----------+



In [59]:
spark.sql("select * from custstate1").show()

+------+----------+
|cstate|statecount|
+------+----------+
|    AZ|        19|
|    NJ|        19|
|    VA|        14|
|    MI|        28|
|    NV|        16|
|    CA|       187|
|    NC|        19|
|    MD|        19|
|    MO|        13|
|    IL|        40|
|    OH|        29|
|    TN|        12|
|    PA|        28|
|    NY|        79|
|    TX|        62|
|    GA|        14|
|    MA|        12|
|    FL|        32|
|    CO|        12|
|    PR|       505|
+------+----------+
only showing top 20 rows



spark.sql("CREATE DATABASE IF NOT EXISTS retaildb")

cStateCount50.coalesce(1).write.saveAsTable("retaildb.cStateCount50")

prd200.coalesce(1).write.saveAsTable("retaildb.prd200")